# Submission File

In [8]:
import cv2
import torch
import numpy as np
import pandas as pd
import mlflow

from pathlib import Path
from tqdm.notebook import tqdm
from einops import rearrange

from eda.predict import predict
from utils.extract import convert_to_rle, CLASS_MAPPING

# IO

In [2]:
def load_image(image_path: Path) -> torch.Tensor:
    img = cv2.imread(str(image_path), cv2.IMREAD_UNCHANGED)
    img = np.asarray(img, np.float32)
    img = torch.from_numpy(img)
    
    return img

# Load Pre-Trained Model

In [3]:
logged_model = "runs:/54db63cc351242399e8fc208d55e0ed7/model"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = mlflow.pytorch.load_model(logged_model)
model = model.to(device)
model

UNet(
  (down): ModuleList(
    (0): DoubleConv(
      (norm): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (act1): GELU()
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (act2): GELU()
    )
    (1): DoubleConv(
      (norm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (act1): GELU()
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (act2): GELU()
    )
    (2): DoubleConv(
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (act1): GELU()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (act2): GELU()
    )
    (3): Do

In [28]:
DATA_DIR = Path("/kaggle/input/uw-madison-gi-tract-image-segmentation")
TEST_DIR = DATA_DIR / "test"

image_paths = list(TEST_DIR.rglob("*.png"))

# Use local images for testing
if len(image_paths) == 0:
    image_paths = list(Path("raw_dataset").rglob("*.png"))
    
assert len(image_paths) > 0, "Could not find any images"

frames = []

n = 0
for path in tqdm(image_paths, total=len(image_paths)):
    slice_number = path.stem[:10]
    case_day = path.parent.parent.name
    index = f"{case_day}_{slice_number}"

    image = load_image(path)
    image = image.to(device)
    probs = predict(model, image)
    mask = np.argmax(probs.cpu().detach().numpy(), axis=0)
    rle = convert_to_rle(mask, index)
    frames.append(rle)

  0%|          | 0/38496 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [27]:
sub = pd.concat(frames).reset_index(drop=True)
sub.to_csv("submission.csv", index=False)
sub

,id,class,predicted
0,case131_day19_slice_0060,large_bowel,None
1,case131_day19_slice_0060,small_bowel,None
2,case131_day19_slice_0060,stomach,None
3,case131_day19_slice_0061,large_bowel,None
4,case131_day19_slice_0061,small_bowel,None
...,...,...,...
298,case131_day19_slice_0139,small_bowel,None
299,case131_day19_slice_0139,stomach,None
300,case131_day19_slice_0100,large_bowel,None
301,case131_day19_slice_0100,small_bowel,None
